<a href="https://colab.research.google.com/github/machiwao/CCTHESS1-CCTHESS2-Dev-and-Docs/blob/coli/16_17_scraping_v1.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install earthengine-api --upgrade
# optional, but useful for interactive maps
!pip install geemap --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.2/463.2 kB 29.6 MB/s eta 0:00:00
  Attempting uninstall: earthengine-api
    Found existing installation: earthengine-api 1.5.24
    Uninstalling earthengine-api-1.5.24:
      Successfully uninstalled earthengine-api-1.5.24
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 631.5/631.5 kB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 84.6 MB/s eta 0:00:00
  Attempting uninstall: geemap
    Found existing installation: geemap 0.35.3
    Uninstalling geemap-0.35.3:
      Successfully uninstalled geemap-0.35.3


# New Import Section


In [4]:
import ee, pandas as pd
import geemap

# Authenticate

In [14]:
ee.Authenticate()
ee.Initialize(project='data-scrapping-470316') #name ng project niyo sa earth engine

# click niyo yung left side ng malapit sa dp
# tapos project info
# then udner Cloud Project ayun i-copy paste


# Load ERA5-Land and Bands

In [6]:
# ERA5-Land Daily Aggregates dataset
dataset = ee.ImageCollection("ECMWF/ERA5/DAILY") \
  .filterDate('2016-03-01', '2017-05-31')

# Print available variables (bands)
bands = dataset.first().bandNames().getInfo()
print("Available Bands:", bands)

Available Bands: ['mean_2m_air_temperature', 'minimum_2m_air_temperature', 'maximum_2m_air_temperature', 'dewpoint_2m_temperature', 'total_precipitation', 'surface_pressure', 'mean_sea_level_pressure', 'u_component_of_wind_10m', 'v_component_of_wind_10m']


# Coordinates


In [20]:
#Lon, Lat
# point = ee.Geometry.Point([12.8797, 121.7740]) if the whole ph

# Geometry Point (manila)
manila = ee.Geometry.Point([14.5995, 120.9842]).buffer(31000)

# Extract All Bands


In [21]:
def extract(img):
  values = img.reduceRegion(
      reducer=ee.Reducer.first(),
      geometry=manila,
      scale=31000, # ERA5 resolution
      maxPixels=1e13
  )
  return ee.Feature(None, values).set("date", img.date().format("YYYY-MM-dd"))


# Apply across all days

In [22]:
fc = dataset.map(extract)

# Get info into Python

In [23]:
dates = fc.aggregate_array("date").getInfo()
bands = dataset.first().bandNames().getInfo()
band_data = {band: fc.aggregate_array(band).getInfo()for band in bands}

# Dataframe

In [24]:
df = pd.DataFrame({"date": dates})
for band in bands:
    df[band] = band_data[band]

# Convert to Kelvin to Celcius

In [25]:
temp_vars = [b for b in bands if 'temperature' in b]
for t in temp_vars:
  df[t] = df[t] - 273.15

# Download CSV

In [27]:
from google.colab import files

filename = "ERA5_Daily_MarMay_2016_2017_Manila.csv"
df.to_csv(filename, index=False)

# --- Download ---
files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#

In [26]:
df.head()

,date,mean_2m_air_temperature,minimum_2m_air_temperature,maximum_2m_air_temperature,dewpoint_2m_temperature,total_precipitation,surface_pressure,mean_sea_level_pressure,u_component_of_wind_10m,v_component_of_wind_10m
0,2016-03-01,1.232721,-0.141943,2.264276,-2.640967,0.001277,99426.250000,99426.710938,-10.517670,-2.563542
1,2016-03-02,1.243890,0.612390,1.617944,-0.616034,0.000079,100350.687500,100351.101562,-6.721692,4.743433
2,2016-03-03,-0.179083,-0.913367,0.857690,-2.267584,0.000000,100782.195312,100782.617188,-5.808779,-4.276811
3,2016-03-04,-1.730902,-3.437109,-0.436499,-4.116827,0.000076,100544.335938,100544.820312,-3.827953,-9.852751
4,2016-03-05,-6.294440,-7.456366,-3.845679,-9.249487,0.001306,99560.023438,99560.500000,-0.716159,-12.306160
